### Logistic Regression Tuning

In [ ]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def objective_logistic_regression(trial):
    # Suggest hyperparameters
    penalty = trial.suggest_categorical("penalty", [None, "l2"])  # Regularization type
    max_iter = trial.suggest_int("max_iter", 200, 1000, step=50)  # Number of iterations

    # Train Logistic Regression model
    model = LogisticRegression(penalty=penalty, max_iter=max_iter, random_state=1)

    # Flatten input features
    X_train_flattened = X_train.reshape(X_train.shape[0], -1)
    X_test_flattened = X_test.reshape(X_test.shape[0], -1)

    # Fit model
    model.fit(X_train_flattened, y_train)

    # Predict on test set
    y_pred = model.predict(X_test_flattened)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy  # Optuna will maximize this

# Run Optuna optimization
study_logistic = optuna.create_study(direction="maximize")  # We want to maximize accuracy
study_logistic.optimize(objective_logistic_regression, n_trials=50)

# Print best hyperparameters
print("Best hyperparameters:", study_logistic.best_params)


### FCNN tuning

In [ ]:
import optuna
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def objective(trial):
    # Suggest hyperparameters
    n_layers = trial.suggest_int("n_layers", 1, 3)  # Number of hidden layers
    layer_sizes = [trial.suggest_int(f"n_units_l{i}", 16, 256, step=16) for i in range(n_layers)]
    alpha = trial.suggest_float("alpha", 1e-5, 1e-1, log=True)  # L2 regularization
    max_iter = trial.suggest_int("max_iter", 200, 500, step=50)  # Number of iterations

    # Flatten input features
    X_train_flattened = X_train.reshape(X_train.shape[0], -1)
    X_test_flattened = X_test.reshape(X_test.shape[0], -1)

    # Create MLP model with dynamic architecture
    model = MLPClassifier(hidden_layer_sizes=tuple(layer_sizes), 
                          alpha=alpha, 
                          max_iter=max_iter, 
                          random_state=1)

    # Train model
    model.fit(X_train_flattened, y_train)

    # Predict on test set
    y_pred = model.predict(X_test_flattened)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy  # Optuna will maximize this

# Run Optuna optimization
study = optuna.create_study(direction="maximize")  # We want to maximize accuracy
study.optimize(objective, n_trials=50)

# Print best hyperparameters
print("Best hyperparameters:", study.best_params)


### TNN Tuning

In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

def objective(trial, X_train, X_test, X_val, y_train, y_test, y_val, num_of_classes, epochs, patience=50):
    # Suggest hyperparameters
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    
    train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    val_dataset = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))

    trainloader = DataLoader(train_dataset, shuffle=True, batch_size=256)
    valloader = DataLoader(val_dataset, shuffle=False, batch_size=256)
    
    model = TensorNet(X_train[0].shape, num_of_classes=num_of_classes)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    best_val_loss = 999
    patience_counter = 0
    best_model_state = None

    # Training loop with validation
    for epoch in range(epochs + 1):
        model.train()
        total_loss = 0
        for windows, labels in trainloader:
            predictions = model(windows)
            loss = criterion(predictions, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Compute validation accuracy
        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        with torch.no_grad():
            for windows, labels in valloader:
                predictions = model(windows)
                loss = criterion(predictions, labels)
                val_loss += loss.item()
        
        val_loss /= len(valloader)
        
        #if epoch % 10 == 0:
            # print(f'Epoch [{epoch}/{epochs}], Loss: {total_loss/len(trainloader):.4f}, Val Loss: {val_loss:.4f}')
        
        # Check if the current model is the best
        if val_loss < best_val_loss or best_val_loss is None:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()  # Save the best model state
            patience_counter = 0  # Reset counter if loss improves
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}, best val loss: {best_val_loss:.4f}")
            break
    
    # Load the best model before testing
    if best_model_state:
        model.load_state_dict(best_model_state)

    # Test the model
    test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
    testloader = DataLoader(test_dataset, shuffle=False, batch_size=256)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for windows, labels in testloader:
            predictions = model(windows)
            _, predicted = torch.max(predictions.data, 1)
            _, ground_truth = torch.max(labels.data, 1)
            total += labels.size(0)
            correct += (predicted == ground_truth).sum().item()

    accuracy_test = correct / total
    print(f'Test Accuracy: {100 * accuracy_test:.2f}%')
    
    return accuracy_test

def tune_tnn(X_train, X_test, X_val, y_train, y_test, y_val, num_of_classes, epochs, n_trials=50):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X_train, X_test, X_val, y_train, y_test, y_val, num_of_classes, epochs), n_trials=n_trials)
    
    print("Best hyperparameters:", study.best_params)
    return study.best_params


best_params = tune_tnn(X_train, X_test, X_val, y_train, y_test, y_val, 16, 1000, n_trials=50)
print("Optimized Hyperparameters:", best_params)